In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import gc 
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
train = pd.read_csv('../input/g-research-crypto-forecasting/train.csv')
print(train.shape)
train.head()

In [ ]:
train.isna().sum()

In [ ]:
supplemetal_train = pd.read_csv('../input/g-research-crypto-forecasting/supplemental_train.csv')
supplemetal_train

In [ ]:
supplemetal_train.isna().sum()

In [ ]:
cat_train = pd.concat([train,supplemetal_train])
cat_train

In [ ]:
del train,supplemetal_train
gc.collect()

In [ ]:
asset_detail = pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv')
asset_detail

In [ ]:
train_set = cat_train.merge(asset_detail,on='Asset_ID',how='left')
train_set

In [ ]:
del cat_train
gc.collect()

In [ ]:
train_set.isna().sum()

In [ ]:
train_set.dropna(inplace=True)

In [ ]:
gc.collect()

In [ ]:
test = pd.read_csv('../input/g-research-crypto-forecasting/example_test.csv')
print(test.shape)
test.head()

In [ ]:
test_set = test.merge(asset_detail,on='Asset_ID',how='left')
test_set.head()

In [ ]:
submission = pd.read_csv('../input/g-research-crypto-forecasting/example_sample_submission.csv')
print(submission.shape)
submission.head()

In [ ]:
train_set.keys()

In [ ]:
# corr_data = train_set[train_set.keys()]
# cols = corr_data.corr().nlargest(len(corr_data.keys()),'Target')['Target'].index
# colormap = plt.cm.PuBu
# print(cols)
# cm = np.corrcoef(train_set[cols].values.T)
# f, ax = plt.subplots(figsize=(15,8))
# heatmap = sns.heatmap(cm,vmax=1,square=True,annot=True,cmap=colormap,xticklabels=cols.values,
#                       linewidths=0.1,linecolor='white',yticklabels=cols.values)


In [ ]:
# del corr_data,cm
# gc.collect()

In [ ]:
def VMA(data):
    data['VMA'] =  data['VWAP']/data['Volume']
    data['VC'] = data['Close']/data['Volume']
    return data

In [ ]:
train_set = VMA(train_set)
test_set = VMA(test_set)

In [ ]:
gc.collect()

In [ ]:
target = train_set.pop('Target')
print(train_set.shape,target.shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,BatchNormalization,Dropout,GaussianNoise
from tensorflow.keras.activations import swish
from tensorflow.keras.models import Sequential


In [ ]:
train_set.drop(['Asset_ID','Asset_Name','timestamp'],axis=1,inplace=True)
test_set.drop(['Asset_ID','Asset_Name','timestamp'],axis=1,inplace=True)

In [ ]:
train_set.head()

In [ ]:
train_set = train_set.astype(np.float16)

In [ ]:
gc.collect()

In [ ]:
train_set.info()

In [ ]:
gc.collect()

In [ ]:
test_set.head()

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
models = []
kf = KFold(n_splits=5,shuffle=True)
for train_idx,valid_idx in kf.split(train_set,target):
    train_x = train_set.values[train_idx]
    train_y = target.values[train_idx]
    val_x = train_set.values[valid_idx]
    val_y = target.values[valid_idx]
    
    model = LGBMRegressor(objective='regression',
                         max_depth=7,
                         n_estimators=3000,
                         learning_rate=0.05,
                         num_leaves=31,
                         n_jobs=-1,
                         device='gpu')
    model.fit(train_x,train_y,
             eval_set=[(val_x,val_y)],
             eval_metric=['rmse'],
             early_stopping_rounds=300,
             verbose=100
             )
    models.append(model)

In [ ]:
del train_set,target
gc.collect()

In [ ]:
test_set.drop(['group_num','row_id'],axis=1,inplace=True)

In [ ]:
prediction = []
for model in models:
    pred = model.predict(test_set.values)
    prediction.append(pred)
result = np.mean(prediction,axis=0)

In [ ]:
result

In [ ]:
submission['Target'] = result

In [ ]:
submission

In [ ]:
# model = Sequential()
# model.add(Dense(256,input_shape=(11,),activation='swish',kernel_initializer='random_normal'))
# model.add(BatchNormalization())
# model.add(Dropout(0.25))
# model.add(Dense(512,activation='swish'))
# model.add(BatchNormalization())
# model.add(Dropout(0.25))
# model.add(Dense(256,activation='swish'))
# model.add(BatchNormalization())
# model.add(Dropout(0.25))
# model.add(Dense(1))
# model.summary()

In [ ]:
# model.compile(optimizer=tf.keras.optimizers.Adamax(learning_rate=0.0003),
#              loss=tf.keras.metrics.MeanSquaredError(),
#              metrics=['loss','val_loss'])

In [ ]:
# earlystopping = tf.keras.callbacks.EarlyStopping(patience=5)
# callbacks = tf.keras.callbacks.ModelCheckpoint('swish_mode.h5',monitor='val_loss')
# scheduler = tf.keras.callbacks.ReduceLROnPlateau(
#     monitor='val_loss',
#     factor=0.6,
#     patience=10,
#     verbose=1,
#     mode='auto',
#     min_lr=0.0001,
#     cooldown=5
#     )

In [ ]:
model.fit(x_train,y_train,validation_data=(x_val,y_val),
                    callbacks=[callbacks,earlystopping,scheduler],
                   epochs=50,batch_size=128)